# ACOPF Convergence Debug

A MATPOWER format ACOPF problem failed to converge, and this notebook compares the results across several tools.

Following tools need manual installation:

- gurobi-optimods, https://github.com/Gurobi/gurobi-optimods
- pypower, https://github.com/rwl/PYPOWER
- pandapower, https://github.com/e2nIEE/pandapower

In [1]:
import ams

import pypower.runopf

import pandapower as pdp

from gurobi_optimods import opf

from scipy.io import savemat

In [2]:
!ams misc --version

Python   3.12.0
ams      1.0.9
andes    1.9.3.post32+gddc308cb
numpy    1.26.4
cvxpy    1.6.0
solvers  CLARABEL, ECOS, ECOS_BB,
         GUROBI, HIGHS, MOSEK, OSQP,
         PIQP, SCIP, SCIPY, SCS


In [3]:
%matplotlib inline

In [4]:
ams.config_logger(stream_level=20)

## AMS

AMS failed for the ACOPF.

After slightly reduce the reactive load, it can converge.

In [5]:
sp = ams.load("./check4.m",
              setup=True,
              no_output=True,)

pq = sp.PQ.idx.v
q0 = sp.PQ.get(src='q0', attr='v', idx=pq)
sp.PQ.set(src='q0', attr='v', idx=pq, value=0.95*q0)

sp.ACOPF.run()

Parsing input file "./check4.m"...
% Converted by AMS 1.0.9
Input file parsed in 0.2184 seconds.
System set up in 0.0087 seconds.
Building system matrices
Parsing OModel for <ACOPF>
Evaluating OModel for <ACOPF>
Finalizing OModel for <ACOPF>
<ACOPF> initialized in 0.7100 seconds.
<ACOPF> solved in 53.0608 seconds, converged in 58 iterations with PYPOWER-PIPS.


In [6]:
mpc = ams.io.matpower.m2mpc("./check4.m")

% Converted by AMS 1.0.9


## gurobi-optimods

Gurobi-optimods converged with the original problem.

In [7]:
savemat('./mpc.mat', {'mpc': mpc})

In [8]:
case = opf.read_case_matpower("mpc.mat")

In [9]:
result = opf.solve_opf(case, opftype="AC")

Set parameter Username
Set parameter LicenseID to value 2617183
Set parameter MIPGap to value 0.001
Set parameter OptimalityTol to value 0.001
Academic license - for non-commercial use only - expires 2026-02-02
Building case data structures from dictionary.
Buses.
    Bus 1797 ID 7389 is the reference bus.
    sumloadPd 56301.405000000086 numPload 1125
    sumloadQd 12223.811000000005
    2000 buses
Branches.
    Numbranches: 3911 active: 3911
Generators.
    Number of generators: 743
    Number of buses with gens: 617
    summaxPg 150046.74999999994 summaxQg 55663.53900000006
Generator cost vectors.
Running ACOPF formulation.
Creating constraints.
  Adding cost definition.
  Adding active power flow definitions.
    7756 active power flow definitions added.
  Adding reactive power flow definitions.
    7756 reactive power flow definitions added.
  Adding constraints stating bus injection = total outgoing power flow.
    4000 balance constraints added.
  Adding injection definition con

## PYPOWER

PYPOWER failed to converge for the original problem.

In [10]:
mpc_sol = pypower.runopf.runopf(mpc)

PYPOWER Version 5.1.18, 10-Apr-2025 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Numerically failed.
Did not converge in 13 iterations.

Did not converge (13.11 seconds)

Objective Function Value = 43701919.12 $/hr
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses           2000     Total Gen Capacity   150046.7       -21722.3 to 55663.5
Generators       743     On-line Capacity     134742.9       -20003.6 to 48373.5
Committed Gens   630     Generation (actual)  58289.7            2045.4
Loads           1125     Load                 56301.4           12223.8
  Fixed         1125       Fixed              56301.4           12223.8
  Dispatchable     0       Dispatchable           0.0 of 0.0        0.0
Shunts           135     Shunt (inj)             -0.0    

## pandapower

pandapower failed to converge, neigher ``init='pf'`` nor ``init='flat'`` worked.

In [11]:
ppn = pdp.converter.from_ppc(mpc, f_hz=60)

/Users/jinningwang/work/miniconda3/envs/amsre/lib/python3.12/site-packages/pandapower/converter/pypower/from_ppc.py:337: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  branch_lookup.loc[is_trafo, "element"] = idx_trafo


In [12]:
pdp.runopp(ppn, init='pf')

OPFNotConverged: Optimal Power Flow did not converge!